#### Setup

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import os
import shutil
import pathlib
import pandas as pd
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt

In [3]:
import librosa
import librosa.display
from pyts.image import GASF, GADF, MTF, RecurrencePlots

#### Download dataset

In [4]:
path = pathlib.Path.home()/'.fastai/data/kaggle_earthquake'

In [5]:
! mkdir -p $path;
! kaggle competitions download -c LANL-Earthquake-Prediction -p $path
! cd $path; if [ ! -f train.csv ]; then unzip -n -q train.csv.zip; fi
! cd $path; if [ ! -d test ]; then unzip -n -q test.zip; mkdir test; mv seg* test; fi

sample_submission.csv: Skipping, found more recently modified local copy (use --force to force download)
test.zip: Skipping, found more recently modified local copy (use --force to force download)
train.csv.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
os.listdir(path)

['sample_submission.csv',
 'test.zip',
 'train.csv.zip',
 'train.csv',
 'X_train.csv',
 'y_train.csv',
 'saved_models',
 'train',
 'train_images',
 'train_images_224',
 'test_images',
 'test_images_224',
 'submission.csv',
 'submission_v6.csv',
 'test']

#### Load training data

In [ ]:
rows = 150_000
train = pd.read_csv(path/'train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [ ]:
segments = int(np.floor(train.shape[0] / rows))
segments

In [ ]:
pd.options.display.precision = 15

In [ ]:
train.head()

#### Write out per segement csv for training set

In [ ]:
train_path = path/'train'
# assume if the dir exists then all the files are there
if not os.path.isdir(train_path):
    os.mkdir(train_path)
        
    for n in tqdm(range(segments)):
        seg = train.iloc[n*rows:n*rows+rows]['acoustic_data']
        seg.to_csv(train_path/f'seg_{n}.csv', header='acoustic_data', index=False)

In [ ]:
len(os.listdir(train_path))

#### Create per segment features

In [ ]:
X_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['ave', 'std', 'max', 'min'])
y_train = pd.DataFrame(index=range(segments), dtype=np.float64,
                       columns=['time_to_failure'])

In [ ]:
for segment in tqdm(range(segments)):
    seg = train.iloc[segment*rows:segment*rows+rows]
    x = seg['acoustic_data'].values
    y = seg['time_to_failure'].values[-1]
    
    y_train.loc[segment, 'time_to_failure'] = y
    
    X_train.loc[segment, 'ave'] = x.mean()
    X_train.loc[segment, 'std'] = x.std()
    X_train.loc[segment, 'max'] = x.max()
    X_train.loc[segment, 'min'] = x.min()

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
# plot histogram of training set
def plot_histogram(targs):
    fig, (ax1) = plt.subplots(1, figsize=(12, 8))
    fig.suptitle("Histograms of segment's time to failure")
    ax1.hist(targs, bins=100); ax1.set_xlabel('targs TTF'); ax1.set_ylabel('No. of segments')

In [ ]:
plot_histogram(np.array(y_train['time_to_failure']))

#### Write out per segment CSVs

In [ ]:
X_train.to_csv(path/'X_train.csv')

In [ ]:
y_train.to_csv(path/'y_train.csv')

#### Visualize acoustic data and spectogram

In [ ]:
# actual sample rate is 4_000_000
sample_rate = 40000

In [ ]:
def plot_segment(segment):
    seg = train.iloc[segment*rows:segment*rows+rows]['acoustic_data']
    plt.figure()
    
    plt.subplot(2, 1, 1)
    plt.plot(seg)
    
    plt.subplot(2, 1, 2)
    seg = np.array(seg, dtype=np.float)
    S = librosa.feature.melspectrogram(y=seg, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max), sr=sample_rate, y_axis='linear', x_axis='time')
    #plt.colorbar(format='%+2.0f dB')

In [ ]:
for i in range(5):
    plot_segment(i)

#### Image generation helper functions

In [28]:
def create_images_from_segment(segment, input_path):
    transforms = ['spectogram', 'gadf', 'gasf']
    
    # load segment
    seg = pd.read_csv(input_path/f'{segment}')
    seg = seg['acoustic_data']
    seg = np.array(seg, dtype=np.float)
        
    for transform in transforms:
        # create plot
        fig = plt.figure(figsize=[0.742, 0.742])
        ax = fig.add_subplot(111)
        ax.axes.get_xaxis().set_visible(False)
        ax.axes.get_yaxis().set_visible(False)
        ax.set_frame_on(False)

        # generate spectogram
        if transform == 'spectogram':
            sample_rate = 40000
            S = librosa.feature.melspectrogram(y=seg, sr=sample_rate)
            librosa.display.specshow(librosa.power_to_db(S, ref=np.max), sr=sample_rate)

        # generate gramian angular difference field
        if transform == 'gadf':
            image_size = 224
            gadf = GADF(image_size)
            img = gadf.fit_transform(seg.reshape(1, -1)).squeeze()
            ax.imshow(img, cmap='viridis', origin='lower')

        # generate gramian angular summation field
        if transform == 'gasf':
            image_size = 224
            gasf = GASF(image_size)
            img = gasf.fit_transform(seg.reshape(1, -1)).squeeze()
            ax.imshow(img, cmap='rainbow', origin='lower')

        # save image
        output_path = pathlib.Path(f'{input_path}_images/{transform}')
        os.makedirs(output_path, exist_ok=True)
        plt.savefig(output_path/f'{segment}'.replace('.csv', '.png'),
                    dpi=400, bbox_inches='tight', pad_inches=0)
        plt.close('all')

#### Generate an image for each segment in training set

In [29]:
train_path = path/'train'
segments = os.listdir(train_path)
for segment in tqdm(segments):
    create_images_from_segment(segment, train_path)

100%|██████████| 4194/4194 [22:48<00:00,  3.10it/s] 


#### Generate an image for each segment in test set

In [30]:
test_path = path/'test'
segments = os.listdir(test_path)
for segment in tqdm(segments):
    create_images_from_segment(segment, test_path)

100%|██████████| 2624/2624 [14:28<00:00,  3.02it/s] 


## IDEAS

#### Consider tempo and beat times as features

In [ ]:
tempo, beat_frames = librosa.beat.beat_track(y=seg1)
beat_times = librosa.frames_to_time(beat_frames)

#### Set correct sample rate at 4MHz

In [ ]:
sample_rate = 4_000_000
segment = 0
seg = train.iloc[segment*rows:segment*rows+rows]['acoustic_data']
seg = np.array(seg, dtype=np.float)
S = librosa.feature.melspectrogram(y=seg, sr=sample_rate)
librosa.display.specshow(librosa.power_to_db(S, ref=np.max), sr=sample_rate, y_axis='linear', x_axis='time')

In [ ]:
S.shape

Throws this errror:
```
/anaconda3/envs/fastai/lib/python3.6/site-packages/librosa/filters.py:284: UserWarning: Empty filters detected in mel frequency basis. Some channels will produce empty responses. Try increasing your sampling rate (and fmax) or reducing n_mels.
  warnings.warn('Empty filters detected in mel frequency basis. '
```

#### Use different type of spectogram more suited to seiesmology

#### Visualize raw acoustic data

In [ ]:
for segment in range(5):
    seg = train.iloc[segment*rows:segment*rows+rows]['acoustic_data']
    plt.figure()
    plt.plot(seg)

#### Visualize spectogram

In [ ]:
# actual sample rate is 4_000_000
sample_rate = 40_000
for segment in range(5):
    seg = train.iloc[segment*rows:segment*rows+rows]['acoustic_data']
    seg = np.array(seg, dtype=np.float)
    S = librosa.feature.melspectrogram(y=seg, sr=sample_rate)
    plt.figure()
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max), sr=sample_rate, y_axis='linear', x_axis='time')

#### Old image generation code for train dataset

In [ ]:
for segment in tqdm(range(segments)):
    # generate plot
    fig = plt.figure(figsize=[0.72,0.72])
    ax = fig.add_subplot(111)
    ax.axes.get_xaxis().set_visible(False)
    ax.axes.get_yaxis().set_visible(False)
    ax.set_frame_on(False)
    # generate spectogram
    seg = train.iloc[segment*rows:segment*rows+rows]['acoustic_data']
    seg = np.array(seg, dtype=np.float)
    S = librosa.feature.melspectrogram(y=seg, sr=sample_rate)
    librosa.display.specshow(librosa.power_to_db(S, ref=np.max), sr=sample_rate)
    # save file
    plt.savefig(img_path/f'seg_{segment}.png', dpi=400, bbox_inches='tight', pad_inches=0)
    plt.close('all')

#### Old resize code

In [ ]:
# pre-resize images to match model input
def resize_images(input_path, output_path, size):
    # create dir
    os.makedirs(output_path, exist_ok=True)
    # duplicate files
    files = os.listdir(input_path)
    for file in files:
        shutil.copyfile(input_path/file, output_path/file)
    # resize with mogrify
    ! cd $output_path; mogrify -resize $size -gravity center -extent $size *png